In [9]:
import onnx
import torch
import torch.nn
import torch.onnx
import numpy as np
import onnxruntime as ort
# import onnxoptimizer
from Model import Transformer, PositionalEncoding
from WordPieceTokenizer import WordPieceTokenizer as Tokenizer

saveFilePath = 'saves/'
tokenizer = Tokenizer(f'{saveFilePath}vocab.txt',do_lower_case=False,strip_accents=False,clean_text=True)
VOCAB_SIZE = tokenizer.get_vocab_size()
MAX_SEQUENCE_LENGTH = 128
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [10]:
save_NN = Transformer(vocab_size=VOCAB_SIZE,embedding_dim=128,hidden_dim=32,output_dim=7,n_layers=2,n_heads=16,dropout_p=0.1,max_len=128,pad_token_id=0)
load_model_path = "Sentiment.pt"
save_NN.load_state_dict(torch.load(load_model_path, map_location=device))
save_NN.eval()
save_NN.to(device)

batch_size = 1
dummy_text_input = torch.randint(0, VOCAB_SIZE, (batch_size, MAX_SEQUENCE_LENGTH),dtype=torch.long).to(device)
dummy_attention_mask = torch.ones(batch_size,MAX_SEQUENCE_LENGTH,dtype=torch.long).to(device)
dummy_attention_mask[:, 64:] = 0

onnx_filename = f"{saveFilePath}onnx/Sentiment_model.onnx"

input_names = ["text_input","attention_mask"]
output_names = ["output_logits"]
dynamic_axes = {
    "text_input": {0: "batch_size", 1: "sequence_length"},
    "attention_mask": {0: "batch_size", 1: "sequence_length"},
    "output_logits": {0: "batch_size"}
}

try:
    torch.onnx.export(save_NN,                           # 변환할 모델
                      (dummy_text_input, dummy_attention_mask), # 모델 입력 (튜플 형태)
                      onnx_filename,                # 저장할 ONNX 파일명
                      export_params=True,           # 모델의 학습된 파라미터 포함 여부
                      opset_version=15,             # ONNX opset 버전 (일반적으로 9, 11, 13, 14, 15, 16, 17)
                                                    # 높을수록 최신 연산을 지원하지만, 런타임 호환성 고려
                      do_constant_folding=True,     # 상수 폴딩 최적화
                      input_names=input_names,      # 입력 노드의 이름
                      output_names=output_names,    # 출력 노드의 이름
                      dynamic_axes=dynamic_axes     # 동적 차원 설정 (선택 사항)
                     )
    print(f"모델이 {onnx_filename}으로 성공적으로 변환되었습니다.")

except Exception as e:
    print(f"ONNX 변환 중 오류 발생: {e}")

모델이 saves/onnx/Sentiment_model.onnx으로 성공적으로 변환되었습니다.


In [11]:
try:
    onnx_model = onnx.load(onnx_filename)
    onnx.checker.check_model(onnx_model) # 모델의 유효성 검사
    print("ONNX 모델 유효성 검사 통과.")
except ImportError:
    print("onnx 패키지가 설치되어 있지 않습니다. pip install onnx로 설치하세요.")
except Exception as e:
    print(f"ONNX 모델 검사 중 오류 발생: {e}")

ONNX 모델 유효성 검사 통과.


In [12]:
# --- 1. PyTorch 모델 준비 (기존 코드에서 사용한 것과 동일) ---
NN = Transformer(vocab_size=VOCAB_SIZE, embedding_dim=128, hidden_dim=32, output_dim=7,
                n_layers=2, n_heads=16, dropout_p=0.1, max_len=128, pad_token_id=0)
NN.load_state_dict(torch.load(load_model_path, map_location='cpu'))
NN.eval()

# --- 2. 더미 입력 준비 (ONNX 변환 시 사용한 것과 동일) ---
batch_size = 1

dummy_text_input_cpu = torch.randint(0, VOCAB_SIZE, (batch_size, MAX_SEQUENCE_LENGTH), dtype=torch.long).cpu()
dummy_attention_mask_cpu = torch.ones(batch_size, MAX_SEQUENCE_LENGTH, dtype=torch.long).cpu()

# --- 3. PyTorch 모델로 추론 ---
with torch.no_grad():
    pytorch_output = NN(dummy_text_input_cpu, dummy_attention_mask_cpu).cpu().numpy()

print(f"PyTorch 모델 출력 형태: {pytorch_output.shape}")
print(f"PyTorch 모델 출력 (상위 5개 값): {pytorch_output[0, :5]}")

# --- 4. ONNX Runtime 세션 생성 ---
onnx_model_path = onnx_filename

try:
    providers = ['CPUExecutionProvider'] 

    ort_session = ort.InferenceSession(onnx_model_path, providers=providers)
    
    onnx_inputs = {inp.name: inp for inp in ort_session.get_inputs()}
    onnx_outputs = {out.name: out for out in ort_session.get_outputs()}

    print(f"\nONNX 모델 입력 이름: {[inp.name for inp in ort_session.get_inputs()]}")
    print(f"ONNX 모델 출력 이름: {[out.name for out in ort_session.get_outputs()]}")

    # --- 5. ONNX Runtime으로 추론 ---
    ort_inputs = {
        "text_input": dummy_text_input_cpu.numpy(),
        "attention_mask": dummy_attention_mask_cpu.numpy()
    }

    ort_output = ort_session.run(
        [onnx_outputs['output_logits'].name], 
        ort_inputs
    )[0]

    print(f"ONNX Runtime 출력 형태: {ort_output.shape}")
    print(f"ONNX Runtime 출력 (상위 5개 값): {ort_output[0, :5]}")

    # --- 6. PyTorch와 ONNX Runtime 출력 비교 ---
    tolerance = 1e-5 
    if np.allclose(pytorch_output, ort_output, atol=tolerance):
        print("\nPyTorch와 ONNX Runtime 모델 출력이 일치합니다! ✅")
    else:
        print("\nPyTorch와 ONNX Runtime 모델 출력이 다릅니다! ❌")
        diff = np.abs(pytorch_output - ort_output)
        print(f"최대 절대 오차: {np.max(diff)}")

except ImportError:
    print("onnxruntime 패키지가 설치되어 있지 않습니다. pip install onnxruntime로 설치하세요.")
except Exception as e:
    print(f"ONNX Runtime 추론 또는 비교 중 오류 발생: {e}")

PyTorch 모델 출력 형태: (1, 7)
PyTorch 모델 출력 (상위 5개 값): [ 0.06537803 -2.1254883   1.4423075   0.34495094 -0.649205  ]

ONNX 모델 입력 이름: ['text_input', 'attention_mask']
ONNX 모델 출력 이름: ['output_logits']
ONNX Runtime 출력 형태: (1, 7)
ONNX Runtime 출력 (상위 5개 값): [ 0.06537812 -2.125488    1.4423069   0.34495088 -0.6492054 ]

PyTorch와 ONNX Runtime 모델 출력이 일치합니다! ✅
